# Fine Tune
Training an existing GPT model to specialize or adapt to a task or specific domain

In [1]:
import os
import openai
import requests

In [2]:
key_location = '/Users/xavier/github/xmorera/ps-generative-ai-developing-applications-python-openai/key/genaikey.txt'

with open(key_location, 'r') as file:
    key = file.readline().strip()

openai.api_key = key

In [3]:
def upload_file(file:str):
    """Takes a prompt as an argument and sends a POST request to the OpenAI API"""
    url = "https://api.openai.com/v1/files"
    headers = { 'Authorization': f'Bearer {key}' }
    data = { "purpose": "fine-tune" }
    files = { "file": open(file, "rb") }
    response = requests.post(url, headers=headers, data=data, files=files, timeout=20)
    if response.status_code == 200:
        result = response.json()
        return result
    print('Request failed with status code:', response.status_code)
    return None


In [4]:
def fine_tuning_model(file_id):
    """Fine Tunes a model using a jsonl file"""
    url = "https://api.openai.com/v1/fine_tuning/jobs"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {key}'
    }
    data = { "training_file": file_id, "model": "gpt-3.5-turbo" }
    response = requests.post(url, headers=headers, json=data, timeout=20)
    if response.status_code == 200:
        result = response.json()
        return result
    print('Request failed with status code:', response.status_code)
    return None

In [5]:
def retrieve_fine_tuning(fine_tune_id):
    """Retrieves a Fine Tune Model"""
    url = f"https://api.openai.com/v1/fine_tuning/jobs/{fine_tune_id}"
    headers = {
        'Authorization': f'Bearer {key}'
    }
    response = requests.get(url, headers=headers, timeout=20)
    if response.status_code == 200:
        result = response.json()
        return result
    print('Request failed with status code:', response.status_code)
    return None

In [6]:
upload_result = upload_file("./data.jsonl")
uploaded_file_id = upload_result["id"]

In [7]:
with open("./data.jsonl") as f: # The with keyword automatically closes the file when you are done
   print(f.read())

{"messages": [{"role": "system", "content": "Abbi is a travel agent chatbot."}, {"role": "user", "content": "Where should I travel in 2023? What are the best destinations to visit?"}, {"role": "assistant", "content": "Here are some fantastic travel destinations for 2023: Poland, Western Australia, Liverpool in England, Charleston in South Carolina, Vilnius in Lithuania, Fiji, Manaus in Brazil, Thessaloniki in Greece, Rwanda, Gothenburg in Sweden, Ras al-Khaimah in the United Arab Emirates, Laos, Gruyères in Switzerland, Minneapolis in Minnesota, Mustang Valley in Nepal, Tanzania, Cairo in Egypt, Naoshima in Japan, Belize, Oaxaca in Mexico, Ottawa in Canada, and Uganda."}]}
{"messages": [{"role": "system", "content": "Abbi is a travel agent chatbot."}, {"role": "user", "content": "Tell me about the attractions in Poland. What can I explore there?"}, {"role": "assistant", "content": "Poland offers a diverse range of attractions. From the historic main square in Krakow to its beautiful fo

In [8]:
fine_tune_model_result = fine_tuning_model(uploaded_file_id) 

fine_tune_model_id = fine_tune_model_result["id"]

In [ ]:
fine_tuned_model = ""

while True:
    fine_tune_object = retrieve_fine_tuning(fine_tune_model_id)
    if fine_tune_object["status"] == "succeeded":
        fine_tuned_model = fine_tune_object["fine_tuned_model"]
        break

In [ ]:
print(str(fine_tuned_model)) 

In [ ]:
completion_response = openai.ChatCompletion.create(
model=fine_tuned_model,
messages=[
    {"role": "system", "content": "Abbi is a travel agent chatbot."},
    {"role": "user", "content": "Where should I travel in 2023?"}
]
)

print(completion_response)